In [1]:
!pip install PyPDF2
!pip install sentence_transformers

In [1]:
import chromadb
chroma_client = chromadb.Client()


In [2]:
import PyPDF2

def extract_text_from_pdf(pdf_file_path):
    text = ""
    with open(pdf_file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

pdf_text = extract_text_from_pdf("ghs.pdf")


In [3]:
def chunk_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

text_chunks = chunk_text(pdf_text)


In [21]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [23]:
query_text = "school policy"

query_embedding = embedding_model.encode([query_text], convert_to_tensor=False)[0]

results = collection.query(
    query_embeddings=[query_embedding.tolist()],  
    n_results=5  
)

for i, result in enumerate(results['documents']):
    print(f"Result {i + 1}:")
    print(f"Document: {result[0]}")
    print(f"Score: {results['distances'][i][0]}")


Result 1:
Document:  to the  Superintendent for approval. 
Both  the request  and the text  are then  presented to the Board  of 
Education  for review.  If you  have  questions  about  instructional 
materials, please consult the appropriate curriculum director.  
 
GRIEVANCE  PROCEDURE  AND COMPLIANCE 
OFFICERS FOR VIOLATIONS AND 
COMPLAINTS   
The Glastonbury Public Schools as a matter of policy 
provides educational opportunities without regard to race, 
creed, color, national origin, religion, gender, sexual o
Score: 0.9250936508178711


In [10]:
import chromadb

client = chromadb.PersistentClient(path='./chorma_store')

collection = client.create_collection("ghs_embeddings")

for i, chunk_embedding in enumerate(embedded_chunks):
    collection.add(
        documents=[text_chunks[i]], 
        embeddings=[chunk_embedding.tolist()],
        ids=[f"chunk_{i}"]
    )


In [24]:
import chromadb

client = chromadb.PersistentClient(path='./chorma_store')

collection = client.get_collection("ghs_embeddings")

query_text = "Computer science"

query_embedding = embedding_model.encode([query_text], convert_to_tensor=False)[0]

results = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=5  
)

for i, result in enumerate(results['documents']):
    print(f"Result {i + 1}:")
    print(f"Document: {result[0]}")
    print(f"Score: {results['distances'][i][0]}")


Result 1:
Document: utions, and the ethical and social implications of 
computing. The course emphasizes both object -oriented and 
imperative problem solving and design using Java language. These techniques represent proven approaches for developing solutions that can scale up from small, simple prob lems to 
large, complex problems. The AP Computer Science A course curriculum is compatible with many CS1 courses in colleges 
and universities. (From the Advanced Placement Program 
Handbook)  
Students who wish to g
Score: 0.8495681285858154


In [27]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

client = chromadb.PersistentClient(path='./chorma_store')

collection = client.get_collection("ghs_embeddings")

query_text = "what is are the computer science at this school?"

query_embedding = embedding_model.encode([query_text], convert_to_tensor=False)[0]

results = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=5 
)

# Print the results
for i, result in enumerate(results['documents']):
    print(f"Result {i + 1}:")
    print(f"Document: {result[0]}")
    context = result[0]


template = """You are an assitant for Glastonbury High School. Answer the question below in a nice and proffessional manner. 
Here is the conversation history {context}
Question: {question}
Answer:
"""

model = OllamaLLM(model='llama3')
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model
result = chain.invoke({'context': context, 'question': query_text})
print(result)

Result 1:
Document: utions, and the ethical and social implications of 
computing. The course emphasizes both object -oriented and 
imperative problem solving and design using Java language. These techniques represent proven approaches for developing solutions that can scale up from small, simple prob lems to 
large, complex problems. The AP Computer Science A course curriculum is compatible with many CS1 courses in colleges 
and universities. (From the Advanced Placement Program 
Handbook)  
Students who wish to g
Thank you for reaching out! At Glastonbury High School, we offer an Advanced Placement Computer Science A course that focuses on teaching students about the fundamentals of computer programming using Java. This course covers a range of topics, including object-oriented and imperative problem-solving techniques, design principles, and algorithm development.

Through this course, students will learn to write efficient, scalable, and well-designed programs to solve complex prob